In [187]:
import torch
from torch import nn
import torch.nn.functional as F

from matplotlib import pyplot as plt
import numpy as np
import helper


ModuleNotFoundError: No module named 'helper'

In [14]:
def activation(x):
    """
        x: torch.Tensor
    """

    return 1/(1+torch.exp(-x))

In [2]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

In [18]:
activation(torch.sum(features*weights)+bias)

tensor([[0.1595]])

In [10]:
weights1 = weights.view(5,1)

In [12]:
inner  = torch.mm(features, weights1)

In [20]:
activation(inner+bias)

tensor([[0.1595]])

In [21]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

In [22]:
features

tensor([[-0.1468,  0.7861,  0.9468]])

In [31]:
features.shape, W1.shape, B1.shape, W2.shape

(torch.Size([1, 3]),
 torch.Size([3, 2]),
 torch.Size([1, 2]),
 torch.Size([2, 1]))

In [32]:
activation(torch.mm(activation(torch.mm(features, W1)+B1), W2)+B2)

tensor([[0.3171]])

In [ ]:
#Numpy to Torch and back

import numpy as np
a = np.random.rand(4,3)

b = torch.from_numpy(a)
b
#The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [4]:
from six.moves import urllib

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [5]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [6]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [12]:
n_input = input_shape  # Number of input units, must match number of input features
n_hidden = 256                    # Number of hidden units 
n_output = 10

W1 = torch.randn(n_input, n_hidden)
W2 = torch.randn(n_hidden, n_output)

B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))g

In [42]:
inputs = images.reshape(64,-1)

In [47]:
h = activation(torch.mm(inputs, W1)+B1)
out = (torch.mm(h, W2)+B2)

In [90]:
def softmax(x):
    proba = torch.exp(x)/torch.sum(torch.exp(x), dim=1).reshape(-1,1)
    return proba

In [91]:
results_batch = softmax(out)

In [110]:
results_batch.shape

torch.Size([64, 10])

In [182]:
#Building a NN

class NeuralNet(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(784,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,10)
        
        
    def forward(self, x):
        
        x = self.fc1(x)
        F.relu_(x)
        
        x = self.fc2(x)
        F.relu_(x)
        
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
    

In [183]:
model = NeuralNet()

In [165]:
model.fc1.bias

Parameter containing:
tensor([ 0.0201, -0.0143,  0.0102,  0.0246,  0.0282,  0.0180, -0.0294,  0.0229,
        -0.0230,  0.0112,  0.0202, -0.0036,  0.0339, -0.0309, -0.0339,  0.0228,
        -0.0308,  0.0110,  0.0284, -0.0025, -0.0343,  0.0174, -0.0169,  0.0200,
         0.0148, -0.0253,  0.0042, -0.0044, -0.0232, -0.0166,  0.0296, -0.0100,
        -0.0235,  0.0222,  0.0039,  0.0302, -0.0131,  0.0133, -0.0226, -0.0326,
        -0.0248,  0.0041,  0.0062, -0.0061, -0.0318,  0.0044, -0.0225,  0.0046,
         0.0230, -0.0001,  0.0083, -0.0069,  0.0014, -0.0019, -0.0050, -0.0153,
        -0.0311,  0.0205, -0.0189,  0.0181,  0.0356, -0.0179, -0.0216,  0.0199,
         0.0262,  0.0218, -0.0258,  0.0203, -0.0247, -0.0159,  0.0195, -0.0301,
         0.0112, -0.0239, -0.0142, -0.0264, -0.0215,  0.0334, -0.0223, -0.0205,
         0.0006, -0.0044,  0.0232,  0.0226,  0.0040, -0.0214,  0.0162, -0.0025,
        -0.0345,  0.0113,  0.0077, -0.0112,  0.0033, -0.0007, -0.0088,  0.0063,
         0.0036, -

In [166]:
model.fc1.bias.data.fill_(0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [167]:
model.fc1.weight.data.normal_(std=0.01)

tensor([[-0.0008, -0.0010, -0.0082,  ..., -0.0015,  0.0109,  0.0185],
        [-0.0002,  0.0150, -0.0163,  ..., -0.0144,  0.0068,  0.0074],
        [-0.0132, -0.0060, -0.0140,  ..., -0.0107,  0.0065, -0.0090],
        ...,
        [ 0.0024, -0.0120,  0.0061,  ..., -0.0012,  0.0271, -0.0097],
        [ 0.0019,  0.0138, -0.0058,  ...,  0.0115, -0.0180, -0.0060],
        [ 0.0214, -0.0057,  0.0159,  ...,  0.0051, -0.0001, -0.0124]])

In [168]:
# Grab some data 
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [169]:
# Resize images into a 1D vector, new shape is (batch size, color channels, image pixels) 
images.resize_(64, 1, 784)
# or images.resize_(images.shape[0], 1, 784) to automatically get batch size

tensor([[[-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.]],

        ...,

        [[-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.]]])

In [188]:
# Forward pass through the network
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]
#helper.view_classify(img.view(1, 28, 28), ps)

In [189]:
img

tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -